In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import re

In [3]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 103.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 108.4 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=9200547dbb6a64f81e72138cf80920dd32c6893088a8f731e989c5278a9743bf
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


In [4]:
df = pd.read_csv('/content/drive/MyDrive/Dataset/train/sarcastic_tweets.csv')

# Pre-processing
remove_symbols = re.compile('[^0-9A-Za-z ]') 

def clean_text(input):
  input = remove_symbols.sub(' ', str(input)) 
  # remove  empty spaces
  re.sub('\s+', '' ,input)
  return input

df['tweet'] = df['tweet'].apply(clean_text)

In [5]:
df_sarcastic = df[df['sarcastic'] == 1 ][['tweet','sarcastic']]
df_rephrase = df[df['sarcastic'] == 0 ][['tweet', 'sarcastic']]

text_sarcastic = df_sarcastic.tweet.values
# labels_sarcastic = df_sarcastic.sarcastic.values

text_rephrase = df_rephrase.tweet.values
# labels_rephrase = df_rephrase.sarcastic.values

In [6]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random

In [7]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased'
    )

In [8]:
# def preprocessing(input_text, tokenizer):
#   '''
#   Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
#     - input_ids: list of token ids
#     - token_type_ids: list of token type ids
#     - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
#   '''
#   return tokenizer.encode_plus(
#                         input_text,
#                         add_special_tokens = True,
#                         max_length = 512,
#                         pad_to_max_length = False,
#                         return_attention_mask = True,
#                         return_tensors = 'pt',
#                         return_token_type_ids = True
#                    )

In [9]:
token_id = []
token_type_id = []
attention_masks = []
labels = []

for element1, element2 in zip(text_sarcastic, text_rephrase):
  encoded_dict_0 = tokenizer(element1, element2, max_length=200,pad_to_max_length=True,return_tensors = 'pt')
  encoded_dict_1 = tokenizer(element2, element1, max_length=200,pad_to_max_length=True,return_tensors = 'pt')
  token_id.append(encoded_dict_0['input_ids'])
  token_id.append(encoded_dict_1['input_ids'])
  token_type_id.append(encoded_dict_0['token_type_ids'])
  token_type_id.append(encoded_dict_1['token_type_ids'])
  attention_masks.append(encoded_dict_0['attention_mask'])
  attention_masks.append(encoded_dict_1['attention_mask'])
  labels.append(0)
  labels.append(1)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [10]:
token_id = torch.cat(token_id, dim = 0)
token_type_id = torch.cat(token_type_id,dim=0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)

In [11]:
# torch.set_printoptions(threshold=10_000)
# len(token_id[0])

In [12]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = True,
    output_hidden_states = True,
)

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(model.parameters(), 
                              lr = 5e-5,
                              eps = 1e-08
                              )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [13]:
batch_size = 16

train_set = TensorDataset(token_id, 
                          attention_masks,
                          token_type_id, 
                          labels)

train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = batch_size
        )

In [14]:
from sklearn.metrics import accuracy_score
def accuracy_func(prediction, true):
  return accuracy_score(true,prediction)

In [15]:
epochs = 2

for _ in trange(epochs, desc = 'Epoch'):
    
    # ========== Training ==========
    
    # Set model to training mode
    model.train()
    
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t for t in batch)
        b_input_ids, b_input_mask,b_token_type_id, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = model(b_input_ids, 
                             token_type_ids = b_token_type_id, 
                             attention_mask = b_input_mask, 
                             labels = b_labels)
        # Backward pass
        train_output.loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1


Epoch: 100%|██████████| 2/2 [14:36<00:00, 438.02s/it]


In [16]:
model_save_name = 'taskc.pt'
path = F"/content/drive/MyDrive/Dataset/{model_save_name}" 
torch.save(model.state_dict(), path)

In [17]:
df_test = pd.read_csv('/content/drive/MyDrive/Dataset/test/task_C_En_test.csv')
text_0 = df_test.text_0.values
text_1 = df_test.text_1.values

test_token_id = []
test_token_type_id = []
test_attention_masks = []
for element1, element2 in zip(text_0, text_1):
  encoded_dict = tokenizer(element1, element2, max_length=200,pad_to_max_length=True,return_tensors = 'pt')
  test_token_id.append(encoded_dict['input_ids'])
  test_token_type_id.append(encoded_dict['token_type_ids'])
  test_attention_masks.append(encoded_dict['attention_mask'])

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


In [ ]:
test_attention_masks

In [25]:
test_token_id = torch.cat(test_token_id, dim = 0)
test_token_type_id = torch.cat(test_token_type_id,dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim = 0)

In [30]:
test_labels = df_test['sarcastic_id']
with torch.no_grad():
  output = model(test_token_id, token_type_ids = test_token_type_id, attention_mask = test_attention_masks)
  logits = output.logits.detach().cpu().numpy()
  preds = np.argmax(logits, axis = 1).flatten()

In [33]:
accuracy_func(test_labels,preds)

0.85

In [35]:
preds.to_csv("/content/drive/MyDrive/Dataset/test/task_c_results")

array([0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0])

In [39]:
pd.DataFrame(preds).to_csv("/content/drive/MyDrive/Dataset/test/task_c_results.csv")

In [43]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(test_labels,preds))

[[93 14]
 [16 77]]


In [44]:
print(classification_report(test_labels, preds))

              precision    recall  f1-score   support

           0       0.85      0.87      0.86       107
           1       0.85      0.83      0.84        93

    accuracy                           0.85       200
   macro avg       0.85      0.85      0.85       200
weighted avg       0.85      0.85      0.85       200

